# Zero Matrix – N-Dimensional

This one poses quite a differnet challenge, as it's extremely difficult to visualize after three dimensions. We also have to consider how we interpret the replacement operation: do we replace with zeroes along each *line* that contains one, or along each *hyperplane*? This is something of an arbitrary choice, but for the sake of the exercise here, I'll look to replace along lines.

In [1]:
import numpy as np
import random
import timeit
from functools import partial
import matplotlib.pyplot as plt
from copy import deepcopy
%matplotlib inline

### Data Generation

First, I need to be able to generate an ND matrix of ones and zeroes with zeroes occurring at a specific probability. This is incredibly easy using Numpy:

In [3]:
test_np = np.random.choice(2, (2,3,4), p=[0.1, 0.9])
test_np

array([[[1, 1, 1, 0],
        [1, 1, 0, 1],
        [1, 1, 0, 1]],

       [[1, 1, 0, 0],
        [1, 1, 1, 1],
        [1, 1, 1, 1]]])

But for kicks, here's a recursive method that does it with Python built-ins + random module:

In [4]:
def gen_ndarray(dims, p_zero):
    """Generates array of provided dimensions filled with randomly assigned 0s and 1s"""

    if isinstance(dims, (list, tuple)) and len(dims) == 1:
        dims = dims[0]
    if isinstance(dims, int):
        return random.choices(range(2), weights=[p_zero, 1-p_zero], k=dims)
    else:
        arr = []
        for _ in range(dims[0]):
            arr.append(gen_ndarray(dims[1:], p_zero))
        return arr

In [5]:
gen_ndarray((2,3,4), 0.1)

[[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0]],
 [[1, 1, 1, 1], [0, 1, 1, 1], [1, 1, 1, 1]]]

In [6]:
%%timeit
gen_ndarray((10,10,10), 0.1)

813 µs ± 31.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
%%timeit
np.random.choice(2, (10,10,10), p=[0.1, 0.9])

55.9 µs ± 1.71 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Not surprisingly, my method is significantly slower than numpy, but it gets the job done.

### Finding Shape, Dimensions, Zero Coordinates

Again, this is something that's incredibly easy if done with Numpy:

In [8]:
print("Shape:", test_np.shape)
print("Dimensions:", len(test_np.shape))

Shape: (2, 3, 4)
Dimensions: 3


In [11]:
# find all zeros, outputs coords by dimension
np.where(test_np == 0)

(array([0, 0, 0, 1, 1]), array([0, 1, 2, 0, 0]), array([3, 2, 2, 2, 3]))

In [140]:
for x,y,z in zip(*np.where(test == 0)):
    print(f"({x},{y},{z})")

(0,1,2)
(1,1,1)
(1,2,2)


But I'll give it a shot with standard code as well...

In [42]:
def detect_shape(arr_in):
    """Return shape of N-dim array"""
    
    test = arr_in
    shape = [len(test)]
    while True:
        test = test[0]
        try:
            shape.append(len(test))
        except TypeError:
            break
    return tuple(shape)


def detect_shape_rec(arr_in, shape=[]):
    """Recursive method to detect shape of N-dim array"""
    
    if not isinstance(arr_in, list):
        return shape
    else:
        shape.append(len(arr_in))
        return detect_shape_rec(arr_in[0], shape)


def get_coords(arr_in, coord=[], zero_coords=[]):
    """Returns coordinates of all zeros from N-dim array"""
    
    # will return false if 1 or list
    if arr_in == 0:
        zero_coords.append(tuple(coord))
        return zero_coords
    if isinstance(arr_in, list):
        for ix, sub in enumerate(arr_in):
            next_coord = coord + [ix]
            get_coords(sub, next_coord)
        return zero_coords

In [24]:
test_arr = gen_ndarray((2,3,4), 0.2)
test_arr

[[[1, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1]],
 [[1, 1, 1, 0], [1, 1, 0, 0], [0, 1, 1, 1]]]

In [34]:
get_coords(test_arr)

[(0, 1, 0), (0, 2, 0), (1, 0, 3), (1, 1, 2), (1, 1, 3), (1, 2, 0)]

In [40]:
detect_shape(test_arr)

(2, 3, 4)

In [43]:
detect_shape_rec(test_arr)

[2, 3, 4]

In [45]:
print("Dimensions:", len(detect_shape(test_arr)))

Dimensions: 3


### Create Class for N-Dim Operations

Coming soon!